In [346]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(3456)

#### Hidden Layer: $h_j = \max{(W^{y}y_{j} + W^{t}t_{j} + W^{h}h_{j-1} + b_{h},0)} $


#### Marker Generation: $P(y_{j+1}=k\mid h_{j}) = \frac{\exp(V_{k,:}^{y}h_{j} + b_{k}^{y})}{\sum_{k=1}^{K} \exp(V_{k,:}^{y}h_{j} + b_{k}^{y})} = \sigma(z)_{k}$ where $\sigma$ is softmax function and $z$ is the vector $ V^{y}h_{j} + b^{y}$ 


#### Conditional Density: $f^{*}(t) = \exp\{{v^{t}}^\top.h_{j} + w^t(t-t_{j}) + b^{t} + \frac{1}{w}\exp({v^{t}}^\top.h_{j} + b^{t}) -\frac{1}{w}\exp({v^{t}}^\top.h_{j} + w^t(t-t_{j}) + b^{t} )\} $

In [505]:
class Rmtpp(nn.Module):
    
    def __init__(self,marker_dim):
        self.N = 1000
        #marker_dim equals to time_dim
        super(Rmtpp, self).__init__()
       
        #linear transformation
        self.lin_op = nn.Linear(3,1) 
        self.vt = nn.Linear(1,1)
        
        #weights
        self.w_t = torch.rand(1)
        self.V_y = torch.rand(marker_dim) #marker dim = number of markers 
        self.b_y = torch.rand(marker_dim) #bias
        
    #compute integral of t*fstart(t) between tj and +infinity using trapezoidal rule
    def next_time(self,tj,hj):
        T = tj + 10 #maximum time
        Deltat = (T-tj)/self.N
        dt = torch.tensor([tj]+[Deltat]*self.N)
        dt = torch.cumsum(dt,dim=0)
        df = dt * self.fstart(dt,tj,hj)
        integral = ((df[1:] + df[:-1])/2) * Deltat
        return torch.sum(integral)
    
    #compute the function fstar
    def fstart(self,t,tj,hj):  
        return (t>tj).float() * torch.exp(self.vt(hj) - torch.exp(self.w_t)*(t-tj) + 1/torch.exp(self.w_t) * (torch.exp(self.vt(hj) - torch.exp(self.w_t)*(t-tj)) - torch.exp(self.vt(hj))))
    
          
        
    def forward(self, time, marker, hidden_state):
        
        #I first compute next time
        tj = time
        #time = self.numerical_mean(time,hidden_state)
        time = self.next_time(time,hidden_state).unsqueeze(-1)
        print("output time",time)
        logfstar = -1 * torch.log(self.fstart(time,tj,hidden_state))
       
        
        #Then next marker distribution
        soft_max = nn.LogSoftmax(dim=0) #softmax of rows
        logprob = -1 * soft_max(self.V_y*hidden_state + self.b_y) 
        print("marker",marker)
        print("hidden",hidden_state)
        #Finally next hidden_state
        input_ = torch.cat((marker, time, hidden_state))
        hidden_state = F.relu(self.lin_op(input_))
        
        return logprob, logfstar, hidden_state
    
    
    def log_likelihood(self,log_time_series_list,log_marker_series_list):
        #time series and marker series are output of the RMTPP network
        #return log_likelihood of all sequences
        lll = 0
        for i in range(len(log_time_series_list)):
            lll += torch.sum(log_time_series_list) + torch.sum(log_marker_series_list)
        return lll       
                
        

In [504]:
a = torch.tensor([0])
torch.cat((a,a,a))

tensor([0, 0, 0])

## Training:

In [506]:
loss = nn.NLLLoss()
learning_rate = 0.0005
rnn = Rmtpp(10)



def train(time,marker):
    #time and marker are list object
    time = torch.tensor(time).unsqueeze(-1)
    marker = torch.tensor(marker).unsqueeze(-1).float()
    loss = 0
    hidden = torch.zeros(1)
    for j in range(len(time)):
        tj = time[j] 
        print("input time",tj) 
        yj = marker[j]
        logprob, logfstar, hidden = rnn(tj, yj, hidden)
        print("-"*10)


time,marker = simulate_timestamps(20)
train(time,marker)  




input time tensor([0.1454])
output time tensor([0.5256], grad_fn=<UnsqueezeBackward0>)
marker tensor([0.])
hidden tensor([0.])
----------
input time tensor([0.1842])
output time tensor([0.5565], grad_fn=<UnsqueezeBackward0>)
marker tensor([1.])
hidden tensor([0.], grad_fn=<ReluBackward>)
----------
input time tensor([0.5911])
output time tensor([0.8804], grad_fn=<UnsqueezeBackward0>)
marker tensor([1.])
hidden tensor([0.], grad_fn=<ReluBackward>)
----------
input time tensor([1.5678])
output time tensor([1.6578], grad_fn=<UnsqueezeBackward0>)
marker tensor([0.])
hidden tensor([0.], grad_fn=<ReluBackward>)
----------
input time tensor([1.5824])
output time tensor([1.6695], grad_fn=<UnsqueezeBackward0>)
marker tensor([0.])
hidden tensor([0.], grad_fn=<ReluBackward>)
----------
input time tensor([1.9706])
output time tensor([1.9785], grad_fn=<UnsqueezeBackward0>)
marker tensor([0.])
hidden tensor([0.], grad_fn=<ReluBackward>)
----------
input time tensor([2.9309])
output time tensor([2.74

# Tick.Hawkes

In [139]:
from tick.plot import plot_point_process
from tick.hawkes import SimuHawkes, HawkesKernelSumExp
import matplotlib.pyplot as plt

#### 1 dimensional Hawkes process simulation using tick

In [140]:
run_time = 40

hawkes = SimuHawkes(n_nodes=1, end_time=run_time, verbose=False, seed=1398)
kernel1 = HawkesKernelSumExp([.1, .2, .1], [1., 3., 7.])
hawkes.set_kernel(0, 0, kernel1)
hawkes.set_baseline(0, 1.)

dt = 0.01
hawkes.track_intensity(dt)
hawkes.simulate()
timestamps = hawkes.timestamps
intensity = hawkes.tracked_intensity
intensity_times = hawkes.intensity_tracked_times

_, ax = plt.subplots(1, 2, figsize=(16, 4))
plot_point_process(hawkes, n_points=50000, t_min=0, max_jumps=20, ax=ax[0])
plot_point_process(hawkes, n_points=50000, t_min=2, t_max=20, ax=ax[1])

<Figure size 1600x400 with 2 Axes>

In [141]:
def simulate_timestamps(end_time):
    # simulation 2 types of event for exemple selling or buying
    
    hawkes = SimuHawkes(n_nodes=2, end_time=end_time, verbose=False, seed=1398)
    kernel = HawkesKernelSumExp([.1, .2, .1], [1., 3., 7.])
    kernel1 = HawkesKernelSumExp([.2, .3, .1], [1., 3., 7.])
    
    hawkes.set_kernel(0, 0, kernel)
    hawkes.set_kernel(0, 1, kernel)
    hawkes.set_kernel(1, 0, kernel)
    hawkes.set_kernel(1, 1, kernel)
    
    hawkes.set_baseline(0, .8)
    hawkes.set_baseline(1, 1.)

    dt = 0.1
    hawkes.track_intensity(dt)
    hawkes.simulate()
    timestamps = hawkes.timestamps
    t0 = timestamps[0]
    t1 = timestamps[1]
    
    t = []
    marker = []
    n0 = len(t0)
    n1 = len(t1)
    i = 0
    j = 0
    while(i<n0 and j<n1):
        if(t0[i]<t1[j]):
            t.append(t0[i])
            marker.append(0)
            i += 1
        else:
            t.append(t1[j])
            marker.append(1)
            j += 1
    if(i==n0):
        for k in range(n0,n1):
            t.append(t1[k])
            marker.append(1)
    else:
        for k in range(n1,n0):
            t.append(t0[k])
            marker.append(0)
        
        
   
    return t,marker


 

In [343]:
simulate_timestamps(end_time=2000)[0]

[0.1453998273553042,
 0.1841744724280548,
 0.5910960339190892,
 1.5677640924338307,
 1.5824299252948109,
 1.9706387517435,
 2.930921868725349,
 3.69765316580875,
 3.828242832851533,
 3.8690278220013226,
 3.9924803129182216,
 4.109933597350456,
 4.2349385311236345,
 4.528839214780729,
 4.8825305589309345,
 4.906513499400947,
 4.973537632659001,
 5.212763731186806,
 5.331627287925645,
 5.400715155736202,
 5.406657026117649,
 5.454771623071175,
 5.456594504809564,
 5.530331730185287,
 5.578008222492138,
 5.776994557096648,
 5.819088834084263,
 5.855813817764325,
 5.8574804773489895,
 5.860791449431528,
 6.1324627680639185,
 6.347360651207286,
 6.972644024344269,
 7.083367739295406,
 7.210046934473388,
 7.371146157026261,
 7.4045067574341275,
 7.4175271740402025,
 7.459357874777104,
 7.563767041208158,
 7.606555100513292,
 7.679454924040601,
 7.773057141798235,
 7.781373919771833,
 7.813868599569604,
 7.836330879915978,
 7.84895889372964,
 7.879810941526391,
 7.881163676710684,
 7.88976214